In [1]:
import pandas as pd
import pickle
from helper_functions import Helper
from typing import List

In [2]:
def formICODfs(df: pd.DataFrame, asset_universe_cm: List[str]) -> tuple:
    # clean up ico data before creating asset-level and macro-level dfs
    df['ico_momtaz'] = 1
    df['asset_cm'] = df.token.str.lower()
    df.loc[df.asset_cm=='grt', 'ico_start'] = '2020-10-21'
    df.loc[df.asset_cm=='atlas', 'ico_start'] = '2021-08-26'
    df.loc[df.asset_cm=='qnt', 'ico_start'] = '2018-06-02'
    df.loc[df.asset_cm=='iotx', 'ico_start'] = '2018-02-15'
    df.loc[df.asset_cm=='orca', 'ico_start'] = '2018-04-16'
    df.loc[df.asset_cm=='gtc', 'ico_start'] = '2021-05-25'
    df.loc[df.asset_cm=='ht', 'ico_start'] = '2018-01-24'
    df.loc[df.asset_cm=='mpl', 'ico_start'] = '2021-04-27'
    df.loc[df.asset_cm=='stx', 'ico_start'] = '2017-11-30'
    df = df[df.ico_start.notnull()].copy()
    df['ico_date_momtaz'] = pd.to_datetime(df['ico_start'], utc=True).dt.tz_localize(None)
    df = df.drop('ico_start', axis=1)
    df = df.sort_values(by=['asset_cm', 'ico_date_momtaz'])
    df = df.drop_duplicates(subset='asset_cm')

    # form asset level data
    ico_df = df.merge(pd.DataFrame(data={'asset_cm': asset_universe_cm}),
        on='asset_cm', how='inner', validate='many_to_one')
    ico_df = ico_df[['asset_cm', 'ico_date_momtaz', 'price_usd', 'ico_momtaz']]

    # clean up asset level price column
    ico_df.loc[ico_df.asset_cm=='ht', 'price_usd'] = '0.64'
    ico_df.loc[ico_df.asset_cm=='stx', 'price_usd'] = '1.103'
    ico_df.loc[ico_df.price_usd.str.contains('ETH') 
        & ico_df.price_usd.notnull(), 'price_usd'] = 'NaN'
    ico_df.loc[ico_df.price_usd.str.contains('STG') 
        & ico_df.price_usd.notnull(), 'price_usd'] = 'NaN'
    ico_df['ico_price_momtaz'] = ico_df.price_usd.astype(float)
    ico_df = ico_df.drop('price_usd', axis=1)
    ico_df.loc[ico_df.asset_cm=='chz', 'ico_price_momtaz'] = 0.03
    ico_df.loc[ico_df.asset_cm=='kai', 'ico_price_momtaz'] = 0.05
    ico_df.loc[ico_df.asset_cm=='lsk', 'ico_price_momtaz'] = 0.08
    ico_df.loc[ico_df.asset_cm=='mpl', 'ico_price_momtaz'] = 0.25
    ico_df.loc[ico_df.asset_cm=='nmr', 'ico_price_momtaz'] = 0.01
    ico_df.loc[ico_df.asset_cm=='stg', 'ico_price_momtaz'] = 0.00075
    ico_df.loc[ico_df.asset_cm=='ocean', 'ico_price_momtaz'] = 0.12
    ico_df.loc[ico_df.asset_cm=='rndr', 'ico_price_momtaz'] = 0.06
    ico_df.loc[ico_df.asset_cm=='uma', 'ico_price_momtaz'] = 0.26

    # final clean of ico asset level data
    ico_df = ico_df[['asset_cm', 'ico_date_momtaz', 'ico_price_momtaz', 'ico_momtaz']]
    ico_df = ico_df.sort_values(by='asset_cm', ignore_index=True)

    # form the macro data
    macro_df = df[['asset_cm', 'ico_date_momtaz', 'ico_momtaz']]
    macro_df = macro_df.groupby('ico_date_momtaz')[['ico_momtaz']].sum()
    macro_df = macro_df[macro_df.index>='2016-07-01']

    start_date_str = '2016-07-01'
    start_date_dt  = pd.to_datetime(start_date_str)
    end_date_str   = '2022-12-31'
    end_date_dt    = pd.to_datetime(end_date_str)
    full_date_range = pd.date_range(start=start_date_dt, end=end_date_dt, freq='1D')

    macro_df = macro_df.reindex(full_date_range)

    macro_df = macro_df.fillna(0)
    macro_df = macro_df.reset_index()
    macro_df = macro_df.rename(columns={'index': 'date', 'ico_momtaz': 'ico_sum_momtaz'})

    return ico_df, macro_df


In [3]:
if __name__ == "__main__":
    # set args
    ICO_IN_FP   = '../data/raw/momtaz.csv'
    ASSET_IN_FP = '../data/clean/asset_universe_dict.pickle'
    ASSET_ICO_OUT_FP = '../data/derived/momtaz_ico_asset.pkl' 
    MACRO_ICO_OUT_FP = '../data/derived/momtaz_ico_macro.pkl' 
    
    # import
    df = pd.read_csv(ICO_IN_FP, sep=';')
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe_cm = Helper.findUniqueAssets(asset_universe_dict)
    
    # clean
    ico_df, macro_df = formICODfs(df, asset_universe_cm)

    # output
    ico_df.to_pickle(ASSET_ICO_OUT_FP)
    macro_df.to_pickle(MACRO_ICO_OUT_FP)